Analyzing Women's Concert Tours: Cleaning and Insights from Kaggle Data

##Project Overview

This project cleans and analyzes a 'dirty' Kaggle dataset on the  highest-grossing concert tours by female artists. The dataset contains issues like inconsistent formatting, missing values, and typos,
  which I address using Python and pandas.
  My goal is to transform this messy data into a clean, analysis-ready format and uncover insights about tour economics,
  showcasing skills in data cleaning, exploratory data analysis (EDA), and visualization.

Let's dive into transforming this chaotic dataset into a polished resource!

Step 1: Loading and Inspecting the Raw Data

First, I load the dataset and inspect its structure to identify issues. The dataset includes columns like Rank, Artist, Tour, Year, Gross, Attendance, and No. of Shows. Common problems include:
  - **Numeric columns** (Gross, Attendance, Shows): Commas, dollar signs, 'N/A', or spaces.
    
  - **Text columns** (Artist, Tour): Mixed casing, extra spaces, or typos.
  - **Year**: Inconsistent formats (e.g., '2016' vs. '2016-2017').
  - **Missing values** and potential duplicates.
   
    The code below loads the data and displays its initial state.

In [30]:
import pandas as pd
import numpy as np

df_dirty = pd.read_csv('/content/drive/MyDrive/women_concert.csv')

df_dirty.head()

,Rank,Peak,All Time Peak,Actual gross,Adjusted gross (in 2022 dollars),Artist,Tour title,Year(s),Shows,Average gross,Ref.
0,1,1,2,"$780,000,000","$780,000,000",Taylor Swift,The Eras Tour †,2023–2024,56,"$13,928,571",[1]
1,2,1,7[2],"$579,800,000","$579,800,000",Beyoncé,Renaissance World Tour,2023,56,"$10,353,571",[3]
2,3,1[4],2[5],"$411,000,000","$560,622,615",Madonna,Sticky & Sweet Tour ‡[4][a],2008–2009,85,"$4,835,294",[6]
3,4,2[7],10[7],"$397,300,000","$454,751,555",Pink,Beautiful Trauma World Tour,2018–2019,156,"$2,546,795",[7]
4,5,2[4],NaN,"$345,675,146","$402,844,849",Taylor Swift,Reputation Stadium Tour,2018,53,"$6,522,173",[8]


In [31]:
df_dirty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Rank                              20 non-null     int64 
 1   Peak                              9 non-null      object
 2   All Time Peak                     6 non-null      object
 3   Actual gross                      20 non-null     object
 4   Adjusted gross (in 2022 dollars)  20 non-null     object
 5   Artist                            20 non-null     object
 6   Tour title                        20 non-null     object
 7   Year(s)                           20 non-null     object
 8   Shows                             20 non-null     int64 
 9   Average gross                     20 non-null     object
 10  Ref.                              20 non-null     object
dtypes: int64(2), object(9)
memory usage: 1.8+ KB


Step 2: Cleaning the Data
        To make the data analysis-ready, I address each issue systematically
  1. **Rank**: Convert to numeric, handle missing values
 2. **Artist and Tour**: Standardize casing, remove extra spaces.
  3. **Gross and Attendance**: Remove symbols ($, commas), convert to numeric, handle 'N/A'.
  4. **No. of Shows**: Ensure numeric format.
  5. **Year**: Extract start year for consistency.
  6. **Duplicates and Missing Values**: Remove duplicates and rows with critical missing data.
  
  Each step is documented below with code and rationale.

In [32]:
df = df_dirty.copy() # Create a copy to preserve original data
df['Peak'] = df['Peak'].astype(str).str.replace(r'\\[.*\\]', '', regex=True).str.strip()
df['Peak'] = pd.to_numeric(df['Peak'], errors='coerce').fillna(-1)
df['All Time Peak'] = df['All Time Peak'].astype(str).str.replace(r'\\[.*\\]', '', regex=True).str.strip()
df['All Time Peak'] = pd.to_numeric(df['All Time Peak'], errors='coerce').fillna(-1)


df.head()

,Rank,Peak,All Time Peak,Actual gross,Adjusted gross (in 2022 dollars),Artist,Tour title,Year(s),Shows,Average gross,Ref.
0,1,1.0,2.0,"$780,000,000","$780,000,000",Taylor Swift,The Eras Tour †,2023–2024,56,"$13,928,571",[1]
1,2,1.0,-1.0,"$579,800,000","$579,800,000",Beyoncé,Renaissance World Tour,2023,56,"$10,353,571",[3]
2,3,-1.0,-1.0,"$411,000,000","$560,622,615",Madonna,Sticky & Sweet Tour ‡[4][a],2008–2009,85,"$4,835,294",[6]
3,4,-1.0,-1.0,"$397,300,000","$454,751,555",Pink,Beautiful Trauma World Tour,2018–2019,156,"$2,546,795",[7]
4,5,-1.0,-1.0,"$345,675,146","$402,844,849",Taylor Swift,Reputation Stadium Tour,2018,53,"$6,522,173",[8]


Note: -1 represents Unknown because those values were empty in the dataset and there's a need to visualize the data. Replacing it with mean, median or mode might skew the data to the corresponding replacement, thereby affecting the accuracy of the result.

In [33]:
cols = ['Actual\xa0gross', 'Adjusted\xa0gross (in 2022 dollars)', 'Average gross']

for col in cols:
    df[col] = (
        df[col]
        .astype(str)                                  # ensure string
        .str.replace(r'[\$,a-zA-Z\[\]]', '', regex=True)  # remove $, letters, brackets
        .str.replace(',', '', regex=False)            # remove commas
        .str.strip()
    )
    df[col] = pd.to_numeric(df[col], errors='coerce')

display(df.head())

,Rank,Peak,All Time Peak,Actual gross,Adjusted gross (in 2022 dollars),Artist,Tour title,Year(s),Shows,Average gross,Ref.
0,1,1.0,2.0,780000000,780000000,Taylor Swift,The Eras Tour †,2023–2024,56,13928571,[1]
1,2,1.0,-1.0,579800000,579800000,Beyoncé,Renaissance World Tour,2023,56,10353571,[3]
2,3,-1.0,-1.0,411000000,560622615,Madonna,Sticky & Sweet Tour ‡[4][a],2008–2009,85,4835294,[6]
3,4,-1.0,-1.0,397300000,454751555,Pink,Beautiful Trauma World Tour,2018–2019,156,2546795,[7]
4,5,-1.0,-1.0,345675146,402844849,Taylor Swift,Reputation Stadium Tour,2018,53,6522173,[8]


In [34]:
df['Artist'] = df['Artist'].str.strip().str.title()
df['Tour title'] = df['Tour title'].str.strip().str.title()

df.head()

,Rank,Peak,All Time Peak,Actual gross,Adjusted gross (in 2022 dollars),Artist,Tour title,Year(s),Shows,Average gross,Ref.
0,1,1.0,2.0,780000000,780000000,Taylor Swift,The Eras Tour †,2023–2024,56,13928571,[1]
1,2,1.0,-1.0,579800000,579800000,Beyoncé,Renaissance World Tour,2023,56,10353571,[3]
2,3,-1.0,-1.0,411000000,560622615,Madonna,Sticky & Sweet Tour ‡[4][A],2008–2009,85,4835294,[6]
3,4,-1.0,-1.0,397300000,454751555,Pink,Beautiful Trauma World Tour,2018–2019,156,2546795,[7]
4,5,-1.0,-1.0,345675146,402844849,Taylor Swift,Reputation Stadium Tour,2018,53,6522173,[8]


In [35]:
df['Start_Year'] = df['Year(s)'].str.split('–').str[0]
df['Start_Year'] = pd.to_numeric(df['Start_Year'], errors='coerce')

df = df.drop(columns=["Year(s)"])
#droped the Year column and only analyzing data with the Start Year column
df.head()

,Rank,Peak,All Time Peak,Actual gross,Adjusted gross (in 2022 dollars),Artist,Tour title,Shows,Average gross,Ref.,Start_Year
0,1,1.0,2.0,780000000,780000000,Taylor Swift,The Eras Tour †,56,13928571,[1],2023
1,2,1.0,-1.0,579800000,579800000,Beyoncé,Renaissance World Tour,56,10353571,[3],2023
2,3,-1.0,-1.0,411000000,560622615,Madonna,Sticky & Sweet Tour ‡[4][A],85,4835294,[6],2008
3,4,-1.0,-1.0,397300000,454751555,Pink,Beautiful Trauma World Tour,156,2546795,[7],2018
4,5,-1.0,-1.0,345675146,402844849,Taylor Swift,Reputation Stadium Tour,53,6522173,[8],2018


In [36]:
df['Shows'] = pd.to_numeric(df['Shows'], errors='coerce')

df.head()

,Rank,Peak,All Time Peak,Actual gross,Adjusted gross (in 2022 dollars),Artist,Tour title,Shows,Average gross,Ref.,Start_Year
0,1,1.0,2.0,780000000,780000000,Taylor Swift,The Eras Tour †,56,13928571,[1],2023
1,2,1.0,-1.0,579800000,579800000,Beyoncé,Renaissance World Tour,56,10353571,[3],2023
2,3,-1.0,-1.0,411000000,560622615,Madonna,Sticky & Sweet Tour ‡[4][A],85,4835294,[6],2008
3,4,-1.0,-1.0,397300000,454751555,Pink,Beautiful Trauma World Tour,156,2546795,[7],2018
4,5,-1.0,-1.0,345675146,402844849,Taylor Swift,Reputation Stadium Tour,53,6522173,[8],2018


In [37]:
df = df.drop(columns=['Ref.'])

df.head()

,Rank,Peak,All Time Peak,Actual gross,Adjusted gross (in 2022 dollars),Artist,Tour title,Shows,Average gross,Start_Year
0,1,1.0,2.0,780000000,780000000,Taylor Swift,The Eras Tour †,56,13928571,2023
1,2,1.0,-1.0,579800000,579800000,Beyoncé,Renaissance World Tour,56,10353571,2023
2,3,-1.0,-1.0,411000000,560622615,Madonna,Sticky & Sweet Tour ‡[4][A],85,4835294,2008
3,4,-1.0,-1.0,397300000,454751555,Pink,Beautiful Trauma World Tour,156,2546795,2018
4,5,-1.0,-1.0,345675146,402844849,Taylor Swift,Reputation Stadium Tour,53,6522173,2018


In [38]:
df = df.drop_duplicates(subset=['Artist', 'Tour title'])
df = df.dropna(subset=['Artist', 'Actual\xa0gross']).reset_index(drop=True)

In [41]:
display(df.head())

,Rank,Peak,All Time Peak,Actual gross,Adjusted gross (in 2022 dollars),Artist,Tour title,Shows,Average gross,Start_Year
0,1,1.0,2.0,780000000,780000000,Taylor Swift,The Eras Tour †,56,13928571,2023
1,2,1.0,-1.0,579800000,579800000,Beyoncé,Renaissance World Tour,56,10353571,2023
2,3,-1.0,-1.0,411000000,560622615,Madonna,Sticky & Sweet Tour ‡[4][A],85,4835294,2008
3,4,-1.0,-1.0,397300000,454751555,Pink,Beautiful Trauma World Tour,156,2546795,2018
4,5,-1.0,-1.0,345675146,402844849,Taylor Swift,Reputation Stadium Tour,53,6522173,2018


In [42]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Rank                              20 non-null     int64  
 1   Peak                              20 non-null     float64
 2   All Time Peak                     20 non-null     float64
 3   Actual gross                      20 non-null     int64  
 4   Adjusted gross (in 2022 dollars)  20 non-null     int64  
 5   Artist                            20 non-null     object 
 6   Tour title                        20 non-null     object 
 7   Shows                             20 non-null     int64  
 8   Average gross                     20 non-null     int64  
 9   Start_Year                        20 non-null     int64  
dtypes: float64(2), int64(6), object(2)
memory usage: 1.7+ KB


None

In [50]:
df.to_csv('cleaned_women_concert.csv', index=False) # Save cleaned data

## Exploratory Data Analysis

I analyze top tours, chart performance vs. gross, and trends over time using interactive Plotly visualizations.

With clean data, I explore key trends to uncover insights:\n",
    "- **Top earners**: Which artists grossed the most?\n",
    "- **Attendance vs. Gross**: Is higher attendance tied to higher revenue?\n",
    "- **Time trends**: How have tour revenues evolved over time?\n",
    "\n",
    "The following code computes summary statistics and creates visualizations."

Summary statistics
  
 Top 3 Artists by Gross

In [45]:
# Summary stats
print('Top 3 Tours by Actual Gross:')
display(df.nlargest(3, 'Actual\xa0gross')[['Artist', 'Tour title', 'Actual\xa0gross', 'Peak']])
print(f'Average Actual Gross: ${df["Actual\xa0gross"].mean():,.0f}')

Top 3 Tours by Actual Gross:


,Artist,Tour title,Actual gross,Peak
0,Taylor Swift,The Eras Tour †,780000000,1.0
1,Beyoncé,Renaissance World Tour,579800000,1.0
2,Madonna,Sticky & Sweet Tour ‡[4][A],411000000,-1.0


Average Actual Gross: $287,950,903


Visualization 1: Top 5 tours by gross

In [47]:
import plotly.express as px

# Plot 1: Top 5 tours by gross
top_tours = df.nlargest(5, 'Actual\xa0gross')
fig1 = px.bar(top_tours, x='Actual\xa0gross', y='Artist', color='Peak',
              title='Top 5 Highest-Grossing Tours',
              labels={'Actual\xa0gross': 'Gross (USD)'},
              hover_data=['Tour title', 'Start_Year'])
fig1.write_html('top_tours.html')
fig1.show()

Visualization 2: Gross vs. Peak

In [49]:
import plotly.express as px

# Plot 2: Gross vs. Peak
fig2 = px.scatter(df, x='Peak', y='Actual\xa0gross', size='Shows', color='Artist',
                  title='Gross vs. Chart Peak',
                  labels={'Peak': 'Chart Peak', 'Actual\xa0gross': 'Gross (USD)'},
                  hover_data=['Tour title'])
fig2.update_layout(xaxis={'autorange': 'reversed'})
fig2.write_html('gross_vs_peak.html')
fig2.show()

## Key Insights

- Taylor Swift’s Eras Tour ($780M) leads, with a #1 chart peak.
- High chart peaks (e.g., #1–2) often align with higher gross, but Pink’s success shows fanbase strength matters.
- Post-2020 Trends: Recent tours show higher per-show revenue despite fewer shows, reflecting post-pandemic demand.

## Conclusion

This project cleaned a messy dataset and revealed trends in concert tour success, showcasing **pandas**, **Plotly**, and **data storytelling**. The cleaned data and plots are saved for further use.